# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import re
import pickle

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk

# punkt is a tokenizer that is used to split text into individual words and sentences.
# wordnet is a lexical database of English words that is used for 
# various NLP tasks such as stemming, lemmatization, and synonym detection.
# nltk.download(['punkt', 'wordnet']);
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from scipy.stats import gmean

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
# load data from database
engine = create_engine('sqlite:///ETL_Preparation.db')
df = pd.read_sql_table('ETL_Preparation', engine)
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.groupby('related').count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,6643,19906,19906,19906,19906,19906,19906,19906,...,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906
2,188,188,132,188,188,188,188,188,188,188,...,188,188,188,188,188,188,188,188,188,188


In [6]:
#  this code is to replace any occurrence of the value 2 in the related column with the value 1.
# because the value 2 'so small' in the 'related' column
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)
df.groupby('related').count() # 19906 + 188 = 20094

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,20094,20094,6775,20094,20094,20094,20094,20094,20094,20094,...,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094


In [7]:
# Extract X and y variables from the data for the modelling
X = df['message']
y = df.iloc[:,4:]


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text, url_place_holder_string="urlplaceholder"):
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    # 'Lemmanitizer' is used to reduce words to the 'root word'
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [9]:
# Build a custom transformer which will extract the starting verb of a sentence
# The StartingVerbExtractor class extends the Scikit-learn BaseEstimator and TransformerMixin classes, 
# which provide the necessary functionality for the transformer to be used in a Scikit-learn pipeline.

# the 'class' extract the starting verb of a sentence,
# creating a new feature for the 'model'

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
  
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)

        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

        return False

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

    # 'return self' because 'something' is a 'tranformer' 
    def fit(self, X, y=None):
        return self
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


pipeline using scikit-learn library. The pipeline consists of two main components:

- FeatureUnion: This component allows combining multiple feature extraction methods. In this example, it includes only one feature extraction method - text_pipeline.

- MultiOutputClassifier: This is a wrapper for multi-output classification algorithms. In this example, it uses the AdaBoostClassifier algorithm for multi-output classification.

The text_pipeline feature extraction method consists of two sub-components:

- CountVectorizer: This is used to convert a collection of text documents to a matrix of token counts.

- TfidfTransformer: This is used to transform a count matrix to a normalized term-frequency times inverse document-frequency representation.

The pipeline is designed to handle multi-output classification problems, meaning that it can predict multiple target variables at the same time. 


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline1.fit(X_train, y_train)
# joblib.dump(pipeline_fitted, 'pipeline_fitted.pkl') # save the 'pipeline_fitted' to a file

# 'train' making 'equations'
# pipeline_fitted = pipeline1.fit(X_train, y_train)
# 1754_SatApril2023, about 2 minutes
# 1915_SatApril2023, about 3 minutes

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
# load the saved pipeline from file
# pipeline_fitted = joblib.load('pipeline_fitted.pkl')

y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      5071
               request       0.76      0.54      0.63      1160
                 offer       0.00      0.00      0.00        32
           aid_related       0.75      0.59      0.66      2769
          medical_help       0.56      0.26      0.36       528
      medical_products       0.61      0.30      0.40       331
     search_and_rescue       0.52      0.19      0.28       181
              security       0.25      0.04      0.07       124
              military       0.61      0.26      0.36       230
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.63      0.68       426
                  food       0.79      0.70      0.74       741
               shelter       0.70      0.53      0.60       568
              clothing       0.62      0.43      0.51        95
                 money       0.57      

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline1.get_params().keys()

parameters_grid1 = {'classifier__estimator__learning_rate': [0.03],
              'classifier__estimator__n_estimators': [13]}


# 'f1_micro' is used as the evaluation metric, which is a common metric used for multiclass classification problems.
#  -1 is used to use all available CPU cores.
cv = GridSearchCV(pipeline1, param_grid=parameters_grid1, scoring='f1_micro', n_jobs=-1)
cv


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7fa85f15b5b0>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.03],
                         'classifier__estimator__n_estimators': [13]},
             scoring='f1_micro')

In [14]:
# (cv) will contain the best hyperparameters found during the search, 
# and other information about the search process.
cv.fit(X_train, y_train)

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_ex

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7fa85f15b5b0>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.03],
                         'classifier__estimator__n_estimators': [13]},
             scoring='f1_micro')

In [15]:
# this doesn't make any sense
print(cv.best_params_)

# Get the prediction values from the 'grid search thing'
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

{'classifier__estimator__learning_rate': 0.03, 'classifier__estimator__n_estimators': 13}


In [16]:
y_prediction_test.shape # (6554, 36) rows, 36 columns
y_prediction_train.shape # (19662, 36) rows, 36 columns

(19662, 36)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
# 'classification_report' for the 'test data'
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5071
               request       0.83      0.13      0.22      1160
                 offer       0.00      0.00      0.00        32
           aid_related       0.79      0.19      0.30      2769
          medical_help       1.00      0.01      0.02       528
      medical_products       1.00      0.01      0.01       331
     search_and_rescue       0.58      0.17      0.26       181
              security       0.00      0.00      0.00       124
              military       0.00      0.00      0.00       230
           child_alone       0.00      0.00      0.00         0
                 water       0.58      0.87      0.70       426
                  food       0.76      0.67      0.71       741
               shelter       0.78      0.29      0.42       568
              clothing       0.00      0.00      0.00        95
                 money       0.00      

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

In [18]:
# 'classification_report' for the 'train data'
print(classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.87     15023
               request       0.86      0.15      0.25      3314
                 offer       0.00      0.00      0.00        86
           aid_related       0.81      0.19      0.31      8091
          medical_help       0.70      0.01      0.02      1556
      medical_products       0.71      0.01      0.01       982
     search_and_rescue       0.66      0.20      0.30       543
              security       0.00      0.00      0.00       347
              military       0.00      0.00      0.00       630
           child_alone       0.00      0.00      0.00         0
                 water       0.58      0.84      0.68      1246
                  food       0.78      0.68      0.73      2182
               shelter       0.86      0.30      0.44      1746
              clothing       0.67      0.01      0.01       310
                 money       0.00      

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
# using pipeline2 which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline2.fit(X_train, y_train)

# 0306_MonApril2023, about 2 minutes

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [20]:
# making prediction/guessing from 'pipeline2'
y_prediction_from_train_data = pipeline_fitted.predict(X_train)
y_prediction_from_test_data = pipeline_fitted.predict(X_test)

In [21]:
# print the 'accuracy report' for the 'train data'
print(classification_report(y_train.values, y_prediction_from_train_data, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88     15064
               request       0.79      0.55      0.65      3345
                 offer       0.26      0.07      0.11        88
           aid_related       0.77      0.60      0.67      8113
          medical_help       0.65      0.28      0.39      1545
      medical_products       0.72      0.34      0.46       971
     search_and_rescue       0.68      0.21      0.32       529
              security       0.56      0.10      0.17       344
              military       0.66      0.38      0.49       640
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.63      0.70      1249
                  food       0.82      0.68      0.74      2202
               shelter       0.79      0.55      0.65      1739
              clothing       0.76      0.49      0.59       313
                 money       0.60      

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

In [22]:
# print the 'accuracy report' for the 'test data'
print(classification_report(y_test.values, y_prediction_from_test_data, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5030
               request       0.77      0.54      0.63      1129
                 offer       0.00      0.00      0.00        30
           aid_related       0.78      0.59      0.67      2747
          medical_help       0.67      0.30      0.41       539
      medical_products       0.66      0.35      0.46       342
     search_and_rescue       0.70      0.22      0.34       195
              security       0.26      0.04      0.07       127
              military       0.58      0.32      0.41       220
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.70      0.74       423
                  food       0.81      0.71      0.76       721
               shelter       0.77      0.57      0.66       575
              clothing       0.63      0.57      0.59        92
                 money       0.59      

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

### 9. Export your model as a pickle file

In [24]:
import os
os.getcwd()

'/workspaces/disaster-response-pipeline-uda/Piplines'

In [25]:
# 'wb' write in 'binary mode'
with open('mo2.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.